In [1]:
!apt-get install openjdk-8-jdk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxxf86dga1 openjdk-8-jdk-headless openjdk-8-jre
  openjdk-8-jre-headless x11-utils
Suggested packages:
  openjdk-8-demo openjdk-8-source visualvm icedtea-8-plugin libnss-mdns
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxxf86dga1 openjdk-8-jdk openjdk-8-jdk-headless
  openjdk-8-jre openjdk-8-jre-headless x11-utils
0 upgraded, 10 newly installed, 0 to remove and 39 not upgraded.
Need to get 40.7 MB of archives.
After this operation,

In [2]:
import os

In [3]:
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64"

In [4]:
!pwd

/content


In [5]:
!echo $JAVA_HOME

/usr/lib/jvm/java-8-openjdk-amd64


In [6]:
!pip install pyspark==2.4.4

     |████████████████████████████████| 215.7MB 61kB/s 
     |████████████████████████████████| 204kB 44.1MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130389 sha256=752b919452a887a8827605ab63aa7cda563512f2249548365ea0b0c4320d61d4
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


# New Section

In [7]:
import pyspark

In [8]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [13]:
myspark = SparkSession.builder \
.master("local[*]") \
.appName("amit") \
.config("spark.executor.memory", "6g") \
.config("spark.driver.memory", "4g") \
.getOrCreate()

In [10]:
!ifconfig

/bin/bash: ifconfig: command not found


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
df = myspark.read.csv("/content/sample_data/california_housing_train.csv",header=True)

In [17]:
df.show(5,False)

+-----------+---------+------------------+-----------+--------------+-----------+----------+-------------+------------------+
|longitude  |latitude |housing_median_age|total_rooms|total_bedrooms|population |households|median_income|median_house_value|
+-----------+---------+------------------+-----------+--------------+-----------+----------+-------------+------------------+
|-114.310000|34.190000|15.000000         |5612.000000|1283.000000   |1015.000000|472.000000|1.493600     |66900.000000      |
|-114.470000|34.400000|19.000000         |7650.000000|1901.000000   |1129.000000|463.000000|1.820000     |80100.000000      |
|-114.560000|33.690000|17.000000         |720.000000 |174.000000    |333.000000 |117.000000|1.650900     |85700.000000      |
|-114.570000|33.640000|14.000000         |1501.000000|337.000000    |515.000000 |226.000000|3.191700     |73400.000000      |
|-114.570000|33.570000|20.000000         |1454.000000|326.000000    |624.000000 |262.000000|1.925000     |65500.000000

In [18]:
df.createOrReplaceTempView("parquetFile")
samplecsv = myspark.sql("SELECT count(*) FROM parquetFile")
samplecsv.show()

+--------+
|count(1)|
+--------+
|   17000|
+--------+



In [20]:
df.count()

17000

In [21]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [22]:
table_schema = StructType([StructField('status', StringType(), True),
                     StructField('txn_id', StringType(), True),
                     StructField('trn_date',TimestampType(), True),
                     StructField('fin_nfin', StringType(), True),
                     StructField('proc_code', StringType(), True),
                     StructField('resp_cpde', StringType(), True),
                     StructField('state', StringType(), True),
                     StructField('mob_no', LongType(), True)])

In [28]:
df1 = myspark.read.csv("/content/drive/My Drive/DATA SCIENCE/Data Science Resources/hive_lecture_new/mb_out_large.csv",schema=table_schema,sep=",", header=None)
# df1 = myspark.read.csv("/content/drive/My Drive/DATA SCIENCE/Data Science Resources/hive_lecture_new/mb_out_large.csv")

In [29]:
df1.show(1)

+---------+------------------+-------------------+--------------+--------------+---------+-----------+----------+
|   status|            txn_id|           trn_date|      fin_nfin|     proc_code|resp_cpde|      state|    mob_no|
+---------+------------------+-------------------+--------------+--------------+---------+-----------+----------+
|COMPLETED|INBbxkrb7677446468|2019-05-26 20:42:26|Non-Financials|balanceInquiry|       00|Maharashtra|9945539882|
+---------+------------------+-------------------+--------------+--------------+---------+-----------+----------+
only showing top 1 row



In [30]:
df1.printSchema()

root
 |-- status: string (nullable = true)
 |-- txn_id: string (nullable = true)
 |-- trn_date: timestamp (nullable = true)
 |-- fin_nfin: string (nullable = true)
 |-- proc_code: string (nullable = true)
 |-- resp_cpde: string (nullable = true)
 |-- state: string (nullable = true)
 |-- mob_no: long (nullable = true)



In [32]:
df1.createOrReplaceTempView("csvfile")
bankdata = myspark.sql("SELECT *,cast(cast(trn_date as DATE) as STRING) as trn_date_part FROM csvfile")
bankdata.show(5)

+---------+------------------+-------------------+--------------+--------------+---------+-----------+----------+-------------+
|   status|            txn_id|           trn_date|      fin_nfin|     proc_code|resp_cpde|      state|    mob_no|trn_date_part|
+---------+------------------+-------------------+--------------+--------------+---------+-----------+----------+-------------+
|COMPLETED|INBbxkrb7677446468|2019-05-26 20:42:26|Non-Financials|balanceInquiry|       00|Maharashtra|9945539882|   2019-05-26|
|COMPLETED|INBpmcbm7874430557|2019-05-17 05:30:54|    Financials|  fundTransfer|       00|Maharashtra|9991940167|   2019-05-17|
|COMPLETED|INBycnbh3838184217|2019-05-19 06:52:02|    Financials|  fundTransfer|       00|Maharashtra|9998664135|   2019-05-19|
|   FAILED|INBlevml0264925092|2019-05-18 05:53:07|    Financials|  fundTransfer|       96|Maharashtra|9997491417|   2019-05-18|
|COMPLETED|INBkovsp0528222721|2019-05-27 19:16:44|Non-Financials|balanceInquiry|       00| Tamil Nadu|99

In [33]:
bankdata.printSchema()

root
 |-- status: string (nullable = true)
 |-- txn_id: string (nullable = true)
 |-- trn_date: timestamp (nullable = true)
 |-- fin_nfin: string (nullable = true)
 |-- proc_code: string (nullable = true)
 |-- resp_cpde: string (nullable = true)
 |-- state: string (nullable = true)
 |-- mob_no: long (nullable = true)
 |-- trn_date_part: string (nullable = true)



In [35]:
#bankdata.write.mode("overwrite").parquet("/content/amit_parquet1/out")
bankdata.write.mode("overwrite").partitionBy("trn_date_part").parquet("/content/amit_parquet1/out1")

In [36]:
bankdata2 = bankdata.repartition(1)

In [37]:
bankdata2.write.mode("overwrite").orc("/content/amit_orc")

In [42]:
parquetFile = myspark.read.parquet("/content/amit_parquet1/out")

In [43]:
parquetFile.show()

+---------+------------------+-------------------+--------------+-----------------+---------+--------------+----------+-------------+
|   status|            txn_id|           trn_date|      fin_nfin|        proc_code|resp_cpde|         state|    mob_no|trn_date_part|
+---------+------------------+-------------------+--------------+-----------------+---------+--------------+----------+-------------+
|COMPLETED|INBbxkrb7677446468|2019-05-26 20:42:26|Non-Financials|   balanceInquiry|       00|   Maharashtra|9945539882|   2019-05-26|
|COMPLETED|INBpmcbm7874430557|2019-05-17 05:30:54|    Financials|     fundTransfer|       00|   Maharashtra|9991940167|   2019-05-17|
|COMPLETED|INBycnbh3838184217|2019-05-19 06:52:02|    Financials|     fundTransfer|       00|   Maharashtra|9998664135|   2019-05-19|
|   FAILED|INBlevml0264925092|2019-05-18 05:53:07|    Financials|     fundTransfer|       96|   Maharashtra|9997491417|   2019-05-18|
|COMPLETED|INBkovsp0528222721|2019-05-27 19:16:44|Non-Financia

In [44]:
parquetFile.cache().count()

999999

In [45]:
parquetFile.count()

999999

In [46]:
parquetFile.filter((col("status")=="COMPLETED") | (col("state")=="Maharashtra")).show()

+---------+------------------+-------------------+--------------+-----------------+---------+--------------+----------+-------------+
|   status|            txn_id|           trn_date|      fin_nfin|        proc_code|resp_cpde|         state|    mob_no|trn_date_part|
+---------+------------------+-------------------+--------------+-----------------+---------+--------------+----------+-------------+
|COMPLETED|INBbxkrb7677446468|2019-05-26 20:42:26|Non-Financials|   balanceInquiry|       00|   Maharashtra|9945539882|   2019-05-26|
|COMPLETED|INBpmcbm7874430557|2019-05-17 05:30:54|    Financials|     fundTransfer|       00|   Maharashtra|9991940167|   2019-05-17|
|COMPLETED|INBycnbh3838184217|2019-05-19 06:52:02|    Financials|     fundTransfer|       00|   Maharashtra|9998664135|   2019-05-19|
|   FAILED|INBlevml0264925092|2019-05-18 05:53:07|    Financials|     fundTransfer|       96|   Maharashtra|9997491417|   2019-05-18|
|COMPLETED|INBkovsp0528222721|2019-05-27 19:16:44|Non-Financia

In [47]:
parquetFile.groupBy(["state","status"]).count().collect()

[Row(state='Rajasthan', status='COMPLETED', count=11100),
 Row(state='Andhra Pradesh', status='FAILED', count=16054),
 Row(state='Tamil Nadu', status='FAILED', count=19802),
 Row(state='Maharashtra', status='FAILED', count=100023),
 Row(state='Maharashtra', status='COMPLETED', count=399703),
 Row(state='Karnataka', status='COMPLETED', count=64246),
 Row(state='Goa', status='FAILED', count=10122),
 Row(state='Madhya Pradesh', status='FAILED', count=4026),
 Row(state='Karnataka', status='FAILED', count=15836),
 Row(state='Andhra Pradesh', status='COMPLETED', count=63938),
 Row(state='Gujarat', status='COMPLETED', count=39928),
 Row(state='Goa', status='COMPLETED', count=40274),
 Row(state='Assam', status='FAILED', count=1008),
 Row(state='West Bengal', status='COMPLETED', count=3892),
 Row(state='Punjab', status='FAILED', count=195),
 Row(state='Haryana', status='FAILED', count=1013),
 Row(state='Assam', status='COMPLETED', count=3950),
 Row(state='Madhya Pradesh', status='COMPLETED', co

In [49]:
parquetFile.createOrReplaceTempView("amit321")
myspark.sql("select state, status,count(*) from amit321 group by state, status").show(5)

+--------------+---------+--------+
|         state|   status|count(1)|
+--------------+---------+--------+
|     Rajasthan|COMPLETED|   11100|
|Andhra Pradesh|   FAILED|   16054|
|    Tamil Nadu|   FAILED|   19802|
|   Maharashtra|   FAILED|  100023|
|   Maharashtra|COMPLETED|  399703|
+--------------+---------+--------+
only showing top 5 rows



In [ ]:
parquetFile.describe()

DataFrame[summary: string, status: string, txn_id: string, fin_nfin: string, proc_code: string, resp_cpde: string, state: string, mob_no: string, trn_date_part: string]

In [ ]:
myspark.sql("select state,status,count(*) from amit321 GROUP BY state,status").show()

+--------------+---------+--------+
|         state|   status|count(1)|
+--------------+---------+--------+
|     Rajasthan|COMPLETED|   11100|
|Andhra Pradesh|   FAILED|   16054|
|    Tamil Nadu|   FAILED|   19802|
|   Maharashtra|   FAILED|  100023|
|   Maharashtra|COMPLETED|  399703|
|     Karnataka|COMPLETED|   64246|
|           Goa|   FAILED|   10122|
|Madhya Pradesh|   FAILED|    4026|
|     Karnataka|   FAILED|   15836|
|Andhra Pradesh|COMPLETED|   63938|
|       Gujarat|COMPLETED|   39928|
|           Goa|COMPLETED|   40274|
|         Assam|   FAILED|    1008|
|   West Bengal|COMPLETED|    3892|
|        Punjab|   FAILED|     195|
|       Haryana|   FAILED|    1013|
|         Assam|COMPLETED|    3950|
|Madhya Pradesh|COMPLETED|   15937|
|       Haryana|COMPLETED|    3884|
|       Gujarat|   FAILED|   10094|
+--------------+---------+--------+
only showing top 20 rows



In [50]:
parquetFile.unpersist()

DataFrame[status: string, txn_id: string, trn_date: timestamp, fin_nfin: string, proc_code: string, resp_cpde: string, state: string, mob_no: bigint, trn_date_part: string]

In [51]:
myspark.stop()

In [52]:
myspark

AttributeError: ignored